<a href="https://colab.research.google.com/github/CarolineAndradeR/Atividade_nao_avaliativas_FIAP/blob/main/atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import numpy as np
import tensorflow as tf
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split

In [102]:
# Download dos recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [103]:
# Carregar dados
dados_review = pd.read_csv('/content/drive/MyDrive/review_clientes_melhores_compras.csv')

In [104]:
# Inicialização do Stemmer
stemmer = PorterStemmer()

In [105]:
# Remoção de pontuação e caracteres especiais, normalização e tokenização
def preprocess_text(text):
# Transfoma todas as letras em minúsculas
    text = text.lower()
# Dividir o texto em palavras individuais (tokens)
    tokens = word_tokenize(text)
# Remoção das letras que não fazem parte do alfabeto
    tokens = [token for token in tokens if token.isalpha()]
# Redução de cada palavra em uma única
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

In [106]:
# Aplicação das etapas de pré-processamento
dados_review['review_cliente'] = dados_review['review_cliente'].apply(preprocess_text)

In [107]:
# Remoção de stopwords
stop_words = set(stopwords.words('portuguese'))
def remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

In [108]:
# Aplicação das etapas de stopwords
dados_review['review_cliente'] = dados_review['review_cliente'].apply(remove_stopwords)

In [109]:
# Divisão dos dados em treinamento e teste
X = dados_review['review_cliente']
y = dados_review['sentimento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score

In [111]:
# Criação de um tokenizador
tokenizer = Tokenizer(num_words=1000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [112]:
# conversão dos textos em números
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [113]:
# Padding para ter sequências do mesmo comprimento
max_length = max([len(sequence) for sequence in X_train_sequences])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

In [114]:
# Construção do modelo
model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [115]:
# Compilação do modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [116]:
# Treinamento do modelo
model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))

Epoch 1/10
1073/1073 [==============================] - 22s 19ms/step - loss: 0.4018 - accuracy: 0.8186 - val_loss: 0.2894 - val_accuracy: 0.8960
Epoch 2/10
1073/1073 [==============================] - 7s 6ms/step - loss: 0.2796 - accuracy: 0.8979 - val_loss: 0.2701 - val_accuracy: 0.9040
Epoch 3/10
1073/1073 [==============================] - 7s 7ms/step - loss: 0.2660 - accuracy: 0.9047 - val_loss: 0.2699 - val_accuracy: 0.9000
Epoch 4/10
1073/1073 [==============================] - 6s 6ms/step - loss: 0.2599 - accuracy: 0.9062 - val_loss: 0.2700 - val_accuracy: 0.9008
Epoch 5/10
1073/1073 [==============================] - 7s 7ms/step - loss: 0.2559 - accuracy: 0.9072 - val_loss: 0.2643 - val_accuracy: 0.9031
Epoch 6/10
1073/1073 [==============================] - 6s 5ms/step - loss: 0.2515 - accuracy: 0.9080 - val_loss: 0.2611 - val_accuracy: 0.9045
Epoch 7/10
1073/1073 [==============================] - 6s 5ms/step - loss: 0.2480 - accuracy: 0.9095 - val_loss: 0.2640 - val_accurac

In [117]:
# Avaliação da acurácia
y_pred_prob = model.predict(X_test_padded)
y_pred = (y_pred_prob > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')

269/269 [==============================] - 0s 2ms/step
Acurácia do modelo: 0.90


In [118]:
# Texto para fazer a previsão
nova_avaliacao = "Péssima compra, não recomento a loja!."

# Pré-processamento do texto
nova_texto_processado = preprocess_text(nova_avaliacao)
nova_texto_tokenizado = tokenizer.texts_to_sequences([nova_texto_processado])
nova_texto_predicao = pad_sequences(nova_texto_tokenizado, maxlen=max_length, padding='post')

# Fazendo a previsão
prediction_prob = model.predict(nova_texto_predicao)[0][0]
prediction_class = (prediction_prob > 0.5).astype(int)

print(f"Probabilidade da classe positiva: {prediction_prob:.4f}")
print(f"Previsão da classe (0 - Negativa, 1 - Positiva): {prediction_class}")

1/1 [==============================] - 0s 20ms/step
Probabilidade da classe positiva: 0.6945
Previsão da classe (0 - Negativa, 1 - Positiva): 1


In [59]:
# Texto para fazer a previsão
nova_avaliacao = "Essa foi uma péssima experiência de compra."

# Pré-processamento do texto
nova_texto_processado = preprocess_text(nova_avaliacao)
nova_texto_tokenizado = tokenizer.texts_to_sequences([nova_texto_processado])
nova_texto_predicao = pad_sequences(nova_texto_tokenizado, maxlen=max_length, padding='post')

# Fazendo a previsão
prediction_prob = model.predict(nova_texto_predicao)[0][0]
prediction_class = (prediction_prob > 0.5).astype(int)

print(f"Probabilidade da classe positiva: {prediction_prob:.4f}")
print(f"Previsão da classe (0 - Negativa, 1 - Positiva): {prediction_class}")

1/1 [==============================] - 0s 31ms/step
Probabilidade da classe positiva: 0.4321
Previsão da classe (0 - Negativa, 1 - Positiva): 0


In [60]:
# Função para fazer previsões
def predict_sentiment(text):
    # Pré-processamento do texto
    text_preprocessed = preprocess_text(text)
    text_tokenized = tokenizer.texts_to_sequences([text_preprocessed])
    text_padded = pad_sequences(text_tokenized, maxlen=max_length, padding='post')

    # Fazendo a previsão
    prediction_prob = model.predict(text_padded)[0][0]
    prediction_class = (prediction_prob > 0.5).astype(int)

    sentiment = "Positiva" if prediction_class == 1 else "Negativa"

    print(f"Avaliação: {text}")
    print(f"Probabilidade da classe positiva: {prediction_prob:.4f}")
    print(f"Previsão da classe ({sentiment}): {prediction_class}")
    print("\n")

# Fazer previsões para avaliações positivas
positive_texts = [
    "Produto de excelente qualidade, recomendo!",
    "Atendimento rápido e eficiente, gostei muito.",
    "Ótimo preço e entrega dentro do prazo."
]

for text in positive_texts:
    predict_sentiment(text)

# Fazer previsões para avaliações negativas
negative_texts = [
    "Produto chegou danificado, péssima experiência.",
    "Péssimo atendimento ao cliente, não recomendo.",
    "Entrega atrasou e ninguém responde às reclamações."
]

for text in negative_texts:
    predict_sentiment(text)

1/1 [==============================] - 0s 20ms/step
Avaliação: Produto de excelente qualidade, recomendo!
Probabilidade da classe positiva: 0.9765
Previsão da classe (Positiva): 1


1/1 [==============================] - 0s 20ms/step
Avaliação: Atendimento rápido e eficiente, gostei muito.
Probabilidade da classe positiva: 0.9275
Previsão da classe (Positiva): 1


1/1 [==============================] - 0s 21ms/step
Avaliação: Ótimo preço e entrega dentro do prazo.
Probabilidade da classe positiva: 0.8843
Previsão da classe (Positiva): 1


1/1 [==============================] - 0s 22ms/step
Avaliação: Produto chegou danificado, péssima experiência.
Probabilidade da classe positiva: 0.5117
Previsão da classe (Positiva): 1


1/1 [==============================] - 0s 22ms/step
Avaliação: Péssimo atendimento ao cliente, não recomendo.
Probabilidade da classe positiva: 0.7098
Previsão da classe (Positiva): 1


1/1 [==============================] - 0s 19ms/step
Avaliação: Entrega atrasou e n